<h3>Задание 8. Задача теории игр.</h3><br>
Игра задана платежной матрицей A.<br>
1. Составьте пару двойственных задач, соответствующую игрокам.<br>
2. Найдите оптимальные стратегии игроков и цену игры.<br>
3. Проверьте результаты вычислений на компьютере.

F(w<sub>1</sub>, w<sub>2</sub>,w<sub>3</sub>) = w<sub>1</sub> + w<sub>2</sub> + w<sub>3</sub> → max<br>
*W = [w<sub>1</sub>, w<sub>2</sub>,w<sub>3</sub>]<br>*
A<sub>1</sub> @ W <= 1,<br>
A<sub>2</sub> @ W <= 1,<br>
A<sub>3</sub> @ W <= 1,<br>
w<sub>1</sub>, w<sub>2</sub>,w<sub>3</sub> >=0.

In [1]:
import numpy as np
from gurobipy import *

In [2]:
#Введите матрицу A размера 3x3
A = np.array([list(map(int,input(f'Введите {i+1} строку (числа через пробел)\n').split())) for i in range(3)])

Введите 1 строку (числа через пробел)
1 2 3
Введите 2 строку (числа через пробел)
5 3 4
Введите 3 строку (числа через пробел)
1 4 0


In [3]:
A

array([[1, 2, 3],
       [5, 3, 4],
       [1, 4, 0]])

In [4]:
var_names = ['w1', 'w2', 'w3']                         
profit = np.array([1, 1, 1])                        # Коэффициенты в целевой функции

n = len(profit)                                    
m = len(A)                                 
model = Model("Matr-Game")
w = model.addVars(n, name=var_names) # заводим переменные в модель
obj = model.setObjective(quicksum(profit[j] * w[j] for j in range(n)), GRB.MAXIMIZE)
con = []
for i in range(m):
    con.append(model.addConstr(quicksum(A[i, j] * w[j] for j in range(n)) <= 1, name=var_names[i])) #добавляем ограничения
model.optimize()

Restricted license - for non-production use only - expires 2023-10-25
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 3 rows, 3 columns and 8 nonzeros
Model fingerprint: 0x29a0ade2
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 3 rows, 3 columns, 8 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.0000000e+30   4.250000e+30   3.000000e+00      0s
       4    3.1250000e-01   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.02 seconds (0.00 work units)
Optimal objective  3.125000000e-01


In [5]:
# выводим параметры модели
W = model.X
print("Оптимальная стратегия W =", model.X, 
      "\nОптимальное значение целевой функции Fmax =", model.ObjVal) 

Оптимальная стратегия W = [0.0, 0.25, 0.0625] 
Оптимальное значение целевой функции Fmax = 0.3125


In [6]:
# Анализ оптимального решения
# оценка выполнения ограничений
print('Заданные ограничения:\n',model.RHS)
print('Реальный расход:\n', [quicksum(A[i, j] * w[j] for j in range(n)).getValue() for i in range(m)])
print('Дефицит:\n',model.Slack)

Заданные ограничения:
 [1.0, 1.0, 1.0]
Реальный расход:
 [0.6875, 1.0, 1.0]
Дефицит:
 [0.3125, 0.0, 0.0]


In [7]:
# Оценка устойчивости коэффициентов целевой функции
# Минимальные и максимальные значения коэффициентов
# в целевой функции, сохраняющие оптимальность
model.printAttr(['X', 'Obj', 'SAObjLow', 'SAObjUp'])


    Variable            X          Obj     SAObjLow      SAObjUp 
----------------------------------------------------------------
          w1            0            1         -inf       1.3125 
          w2         0.25            1         0.75          inf 
          w3       0.0625            1     0.705882      1.33333 


In [8]:
#Оптимальные стратегии второго игрока и цена игры
Y = [y/sum(W) for y in W]
Nu_A = 1/sum(W)
Y, Nu_A

([0.0, 0.8, 0.2], 3.2)

In [9]:
#Решение двойственной задачи (задачи для первого игрока)
U = model.Pi
U

[0.0, 0.25, 0.0625]

In [10]:
#Оптимальные стратегии первого игрока
X = [u/sum(U) for u in U]
X

[0.0, 0.8, 0.2]

In [14]:
print(f'Ответ\n\nX∗ = {X} – оптимальная стратегия I игрока,\nY∗ = {Y} – оптимальная стратегия II игрока,\nνA = {Nu_A} – цена игры.')

Ответ

X∗ = [0.0, 0.8, 0.2] – оптимальная стратегия I игрока,
Y∗ = [0.0, 0.8, 0.2] – оптимальная стратегия II игрока,
νA = 3.2 – цена игры.
